# Math Question Answer Verification Competition

## Starter Code

Borrowed from [official Unsloth implementation](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=MKX_XKs_BNZR)

In [1]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.0/162.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: unsloth 2024.11.6
Uninstalling unsloth-2024.11.6:
  Successfully uninstalled unsloth-2024.11.6
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-q82_g4yq/unsloth_5087dfd6c2ae4b8a8fcea9eaab3ea72f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-q82_g4yq/unsloth_5087dfd6c2ae4b8a8fcea9eaab3ea72f
  Resolved https://github.com/unslothai/unsloth.git to commit d8ff860c842095f4729fdd1d5aedf567a9e2c4da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.6-py3-none-any.whl size=161268 sha256=ad9a1fb9de7de28da68babf3b601ebe4979d23e29045e8c2744e3c8e50819e2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-2p0842y7/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Load model and wrap with LoRA adapters

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "multi_head_attention",
                      "attention_output",
                      "lm_head",
                      "cls_output",
                      "attention_bias",
                     ],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!
Unsloth: Offloading output_embeddings to disk to save VRAM


/usr/local/lib/python3.10/dist-packages/unsloth/models/_utils.py:697: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_location = "cpu", 

Unsloth: Training lm_head in mixed precision to save VRAM


## Competition dataset

In [5]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")

# print and see dataset
dataset['train'][0]

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'question': 'What is the radius of the circle inscribed in triangle $ABC$ if $AB = 22, AC=12,$ and $BC=14$? Express your answer in simplest radical form.',
 'is_correct': True,
 'answer': '3.16227766016838',
 'solution': "The circle is inscribed in a triangle, and we know the sides of the triangle.\nTo use the inradius formula, we need to know the area of the triangle.\nWe can use Heron's formula to calculate the area.\n<llm-code>\nimport math\nfrom sympy import *\n\nAB, AC, BC = 22, 12, 14\n\n# Calculate the semiperimeter and area using Heron's formula\ns = (AB + AC + BC) / 2\nK = sqrt(s * (s - AB) * (s - AC) * (s - BC))\n\nprint(K)\n</llm-code>\n<llm-code-output>\n75.8946638440411\n</llm-code-output>\nLet's now use the formula for the radius of the inscribed circle.\n<llm-code>\nr = K / s\nprint(r)\n</llm-code>\n<llm-code-output>\n3.16227766016838\n</llm-code-output>\nThe answer is \\boxed{3.16227766016838}"}

In [6]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not.
Based on the question, the provided answer, and the explanation, your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.


### Question:
{}

### Answer:
{}

### Explanation:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    solution = examples["solution"]
    output      = examples["is_correct"]
    texts = []
    for instruction, input, sol, output in zip(question, ans, solution, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, sol, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [7]:
# Process the training dataset and generate prompt for each datapoint
train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

# train_dataset_full = dataset['train'].shuffle(seed=42)
# validation_size = int(0.01 * len(train_dataset_full))
# train_dataset = train_dataset_full.select(range(len(train_dataset_full) - validation_size))
# validation_dataset = train_dataset_full.select(range(len(train_dataset_full) - validation_size, len(train_dataset_full)))

# print(f"Main training set size: {len(train_dataset_main)}")
# print(f"Validation set size: {len(validation_dataset)}")

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [8]:
#print a smaple training example
train_dataset['text'][0]

"You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not.\nBased on the question, the provided answer, and the explanation, your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.\n\n\n### Question:\nWhat is the radius of the circle inscribed in triangle $ABC$ if $AB = 22, AC=12,$ and $BC=14$? Express your answer in simplest radical form.\n\n### Answer:\n3.16227766016838\n\n### Explanation:\nThe circle is inscribed in a triangle, and we know the sides of the triangle.\nTo use the inradius formula, we need to know the area of the triangle.\nWe can use Heron's formula to calculate the area.\n<llm-code>\nimport math\nfrom sympy import *\n\nAB, AC, BC = 22, 12, 14\n\n# Calculate the semiperimeter and area using Heron's formula\ns = (AB + AC + BC) / 2\nK = sqrt(s * (s - AB) * (s - AC) * (s - BC))\n\nprint(K)\n</llm-code>\n<llm-code-output>\n75.8946638440411\n</llm-code-output>\nLet's now use t

## SFT

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
#         num_train_epochs = 1, # Set this for 1 full training run.
#         warmup_ratio = 0.1,
        max_steps = 25,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
#         gradient_checkpointing=True,
#         max_grad_norm=0.5,
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

Map (num_proc=4):   0%|          | 0/1000000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
for i in range(8):
  trainer_stats = trainer.train()

# iteration = 0
# while True:
#     trainer_stats = trainer.train()

#     FastLanguageModel.for_inference(model)

#     # Evaluate on validation set
#     val_texts_batch = val_dataset.shuffle(seed=3407).select(range(500))
#     inputs = tokenizer(val_texts_batch['text'], return_tensors="pt", padding=True, truncation=True).to("cuda")
#     outputs = model.generate(**inputs, max_new_tokens=64)
#     predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#     # Calculate validation accuracy
#     correct_preds = sum(1 for pred in predictions if pred.strip().endswith("True"))
#     val_accuracy = correct_preds / len(predictions)

#     iteration += 1
#     print(f"Iteration {iteration}: Validation Accuracy: {val_accuracy:.4f}")

#     if val_accuracy >= 0.7:
#         print("Achieved target accuracy on validation set.")
#         break

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 567,279,616


Step,Training Loss
1,1.584600
2,1.602300
3,1.648400
4,1.419600
5,1.334900
6,1.215800
7,1.238900
8,1.095100
9,0.884500
10,0.824000


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 567,279,616


Step,Training Loss
1,0.681300
2,0.686600
3,0.725800
4,0.558600
5,0.539700
6,0.516100
7,0.719400
8,0.646000
9,0.568700
10,0.562000


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 567,279,616


...

Step,Training Loss
1,0.187000
2,0.208000
3,0.185900
4,0.134500
5,0.130000
6,0.105200
7,0.181300
8,0.153800
9,0.144200
10,0.167300


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 567,279,616


Step,Training Loss
1,0.121500
2,0.138500
3,0.120400
4,0.088000
5,0.103500
6,0.096400
7,0.149000
8,0.103600
9,0.098600
10,0.126500


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 567,279,616


Step,Training Loss
1,0.087000
2,0.100000
3,0.080400
4,0.062800
5,0.069200
6,0.060500
7,0.116300
8,0.105200
9,0.094500
10,0.097900


## inference

In [11]:
# Sample inferene data point
test_dataset = dataset['test']

sample_ques = test_dataset['question'][0]
sample_ans = test_dataset['answer'][0]
sample_solu = test_dataset['solution'][0]


In [12]:
# Running inference on single test
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
input_prompt = prompt.format(
        sample_ques, # ques
        sample_ans, # given answer
        sample_solu,
        "", # output - leave this blank for generation! LLM willl generate is it is True or False
    )

print("Input Promt:\n", input_prompt)
inputs = tokenizer(
[
    input_prompt
], return_tensors = "pt").to("cuda")

input_shape = inputs['input_ids'].shape
input_token_len = input_shape[1] # 1 because of batch
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# you can get the whole generated text by uncommenting the below line
# text_generated = tokenizer.batch_decode([outputs, skip_special_tokens=True)

response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
response

Input Promt:
 You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not.
Based on the question, the provided answer, and the explanation, your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.


### Question:
The Parker family needs to leave the house by 5 pm for a dinner party. Mrs. Parker was waiting to get into the bathroom at 2:30 pm. Her oldest daughter used the bathroom for 45 minutes and her youngest daughter used the bathroom for another 30 minutes. Then her husband used it for 20 minutes. How much time will Mrs. Parker have to use the bathroom to leave on time?

### Answer:
205

### Explanation:
Let's solve this problem using Python code.
<llm-code>
minutes_per_hour = 60
minutes_left_before_5 = 5 * minutes_per_hour
total_time_spent_by_family = 45 + 30 + 20
minutes_before_5_after_family = minutes_left_before_5 - total_time_spent_by_family
minutes_before_5_after_family
</llm-code>
<

['True']

## saving model

In [13]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [14]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Batch Inference


In [15]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not.
Based on the question, the provided answer, and the explanation, your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.


### Question:
{}

### Answer:
{}

### Explanation:
{}

### Output:
{}"""

def formatting_test_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    solution = examples["solution"]
    texts = []
    for instruction, input, sol in zip(question, ans, solution):
        text = prompt.format(instruction, input, sol, "")
        texts.append(text)
    return { "text" : texts, }


test_text_dataset = dataset['test'].map(formatting_test_prompts_func, batched = True,)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [16]:
# Running inference on batch test
import gc

FastLanguageModel.for_inference(model)

test_dataset = test_text_dataset['text']
batch_size = 8
output_label = []

for i in range(0, len(test_dataset), batch_size):
    torch.cuda.empty_cache()
    gc.collect()

    batch = test_dataset[i:i + batch_size]

    # Tokenize batch
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")

    input_lengths = inputs['input_ids'].shape[1]

    with torch.no_grad():
      outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, num_beams=1, early_stopping=True)

    # Extract only the generated part and decode
    text_generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract the last line (True/False) from each generated text
    for text in text_generated:
        output_label.append(text.splitlines()[-1])

    print(f"batch-num: {min(i + batch_size, len(test_dataset))}/{len(test_dataset)}")


batch-num: 8/10000
batch-num: 16/10000
batch-num: 24/10000
batch-num: 32/10000
batch-num: 40/10000
batch-num: 48/10000
batch-num: 56/10000
batch-num: 64/10000
batch-num: 72/10000
batch-num: 80/10000
batch-num: 88/10000
batch-num: 96/10000
batch-num: 104/10000
batch-num: 112/10000
batch-num: 120/10000
batch-num: 128/10000
batch-num: 136/10000
batch-num: 144/10000
batch-num: 152/10000
batch-num: 160/10000
batch-num: 168/10000
batch-num: 176/10000
batch-num: 184/10000
batch-num: 192/10000
batch-num: 200/10000
batch-num: 208/10000
batch-num: 216/10000
batch-num: 224/10000
batch-num: 232/10000
batch-num: 240/10000
batch-num: 248/10000
batch-num: 256/10000
batch-num: 264/10000
batch-num: 272/10000
batch-num: 280/10000
batch-num: 288/10000
batch-num: 296/10000
batch-num: 304/10000
batch-num: 312/10000
batch-num: 320/10000
batch-num: 328/10000
batch-num: 336/10000
batch-num: 344/10000
batch-num: 352/10000
batch-num: 360/10000
batch-num: 368/10000
batch-num: 376/10000
batch-num: 384/10000
batch

In [17]:
# Running inference on batch test

# FastLanguageModel.for_inference(model)

# test_dataset = test_text_dataset['text']
# batch_size = 16
# batch_number = int(len(test_dataset)/batch_size)
# output_label = []

# for i in range(batch_number):
#   print("batch-num:", i)
#   inputs_prompt = []
#   for j in range(batch_size):
#       index = batch_size * i + j
#       inputs_prompt.append(test_dataset[index])

#   inputs = tokenizer(inputs_prompt, return_tensors = "pt", padding=True, truncation=True).to("cuda")
#   outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, num_beams=2)
#   text_generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#   for text in text_generated:
#     output_label.append(text.splitlines()[-1])

In [18]:
print(len(output_label))

import csv

cnt = 0
csv_data = [["ID", "is_correct"]]
for i in range(len(output_label)):
  if output_label[i] == 'True':
    cnt = cnt + 1
    csv_data.append([i, True])
  else:
    csv_data.append([i, False])

# Write to CSV file
with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print("successfully write to data.csv")

10000
successfully write to data.csv


In [19]:
from google.colab import files
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>